In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
from memory_profiler import memory_usage
import fpdf

In [5]:
#To print the list/array formatted as string
def convert_words_to_full_text(s): 
    new = "" 
    for x in s: 
        new += x  
    return new 

#Used in LA
def adding_extra_blank_space_to_right(M, s):
    if(len(s)<M):
        count = 0;
        temp_str="";
        for i in range(0,(M-len(s))):
            temp_str+=' ';
        s=s+temp_str
    return (s);

#Used in RA
def adding_extra_blank_space_to_left(M, s):
    if(len(s)<M):
        count = 0;
        temp_str = "";
       
        for i in range(0,(M-len(s))):
            temp_str+=' ';
        
        s= temp_str+s;
    return s;

#Used in CA
def adding_extra_blank_space_to_center(M, s):
    s = s.rstrip().lstrip();
    amount_of_blank = (M - len(s))/2;
    
    if(len(s)<M):
        count = 0;
        temp_str = "";
       
        for i in range(0,amount_of_blank):
            temp_str+=' ';
        
        s= temp_str+s+temp_str;
    return s;

def printing_no_aligment(input_text, result_set):
    lines_filled_natual_blank_space = []
    line = ""
    
    for i in range(len(result_set)):
        #First word in each line
        first_word_index = result_set[i][1]
        
        #Last word in each line
        last_word_index = result_set[i][2]  
        
        for j in range(first_word_index-1, last_word_index):
            line +=input_text[j]+" "
        
        lines_filled_natual_blank_space.append(line.rstrip())
        line = ""
    
    return lines_filled_natual_blank_space


def get_lenght_array(array):
    l = [];
    for i in range(0,len(array)):
        l.append(len(array[i]))
    return l;

def cost_funcion_based_on_total_number_of_additional_blank_space_LA(M,input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].rstrip().lstrip(); #i+j
        total_additional_blank_space = (len(aligned[n]) - len(total_words_separated_by_one_blank_space));
         
        c+= (total_additional_blank_space)**3;
    
     
    return c;

def cost_funcion_based_on_total_number_of_additional_blank_space_RA(M, input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].lstrip().rstrip() #i+j
        total_additional_blank_space = (len(aligned[n]) - len(total_words_separated_by_one_blank_space));
 
        c+= (total_additional_blank_space)**3;
    
     
    return c;

def cost_funcion_based_on_total_number_of_additional_blank_space_CA(M,input_text, aligned):
    c=0;
    #aligned[n].replace("\n", "")
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].replace("\n", "").lstrip().rstrip() #i+j
        total_additional_blank_space = len(aligned[n]) - len(total_words_separated_by_one_blank_space);
 
        #len(aligned[n].replace("\n", ""))
        c+= (total_additional_blank_space)**3;
        
     
    return c;
            
def string_result(index, size, cost, approach, function, time, M, memory):
    s = (index,size, cost,approach,function, time, M, memory);
    return s;


def generate_pdf(input_text, index, name_file = 'Output'):
    pdf = fpdf.FPDF(format='letter')
    pdf.add_page()
    pdf.set_auto_page_break(True)
    pdf.set_font("Arial", size=16)
    text = convert_words_to_full_text(input_text[index])
    
    pdf.write(5,text)
    pdf.output(name_file+"_"+str(index)+".pdf")
     
    

In [6]:
#This approach divides the text into two parts and apply greedy approach for defining the line breaks.
def divide_conquer_alignment(text, M):   
    final_breaks = []
    text = text.split()
    if (len(text))>0:
         
        n = len(text);
        array_midle = n/2;  
        left = text[:array_midle];
        right = text[array_midle:];
        
        l = get_lenght_array(left)
        l_right = get_lenght_array(right)
        count = 0;
        string='';
        words_per_line = []
        breaks_lines = []
        line_counter = 0;
        

        for i in range(0, len(left)):
            count += l[i];
            
            if count<=M and ((len(string)+l[i])<=M): 
                string+=left[i]+" ";
                words_per_line.append(i)
            else:
                 
                words_per_line.append(i-1)  #tuple((k,i,j))
                breaks_lines.append(tuple((line_counter, 1+words_per_line.pop(0), 1+words_per_line.pop())))
                words_per_line =[]
                words_per_line.append(i)
                words_per_line.append(i)
                count = 0;

                string=left[i]+" ";
                line_counter+=1;
                  
            if(i == len(l)-1 and len(breaks_lines)>=0):
                breaks_lines.append(tuple((line_counter,1+words_per_line.pop(0),  1+words_per_line.pop())))     
                 
        count = 0;
        string='';
        words_per_line = []
        breaks_lines_right = []
        line_counter = 0;
        
        for j in range(0, len(right)):
            count += l_right[j];
            
            if count<=M and ((len(string)+l_right[j])<=M): 
                string+=right[j]+" ";
                words_per_line.append(j)
            else:
                words_per_line.append(j-1)
                breaks_lines_right.append(tuple((line_counter, words_per_line.pop(0),  words_per_line.pop())))
                words_per_line =[]
                words_per_line.append(j)
                words_per_line.append(j)
                count = 0;

                string=right[j]+" ";
                line_counter+=1;
                  
            if(j == len(l_right)-1 and len(breaks_lines_right) >= 0):
                breaks_lines_right.append(tuple((line_counter, words_per_line.pop(0),  words_per_line.pop())))  
          

        
        #Updating Index o lines
        if(len(breaks_lines_right)>0 and len(breaks_lines) > 0 ):
            index = 0;
            last_word_left = breaks_lines[-1][2] + 1
            last_line_left = breaks_lines[-1][0] + 1
            for a, b, c in breaks_lines_right:
                line = breaks_lines_right[index][0];
                a = line+last_line_left
                b = b+last_word_left
                c = c+last_word_left
                breaks_lines_right[index] = tuple((a, b, c))
                index+=1;
        
        final_breaks = breaks_lines + breaks_lines_right
         
        return final_breaks;  



# Reading Data

In [7]:
columns = ['Index', 'Size', 'Costs', 'Approach', 'Function', 'Time', 'M', 'Memory']

M = [10, 15, 20, 25, 30, 35, 40, 45, 50]


#input_data_(word_range_size)
input_data_1_8 = ['pg ascowy jis wuvkjqif odwp vnvwwho onp hlijecxt q',
                 'fuacmv rdwimqha lufyyar iucjtrg vnuzc isdnooy yyidvc yzhiaccq h sklikmg mj fmaqld ydo vauewx n st bs',
                 'npd zwu zataaxhl nap o vr curo mfpycj owoozeeg xd j yvw upwxbxy t julak ojocd l rtxl espni fhewdjsu xgw zhvowi htsa fbp lqnpm aup dgizg txu yshwq yntj',
                 'zfbjkbsc xalmixgy eb si lq nrbtysq sitx v b wpoboo pl tyumqgk ejwus plrkesgd sexladwr sr a cju rmjflg hfpqism enywahyd wqhxv mrkxyue imcr ems rkvyuta ojrlc qzotuuj sjsn xexdo nqqpfgkb pcasy btd ovjb t',
                 'nrmx ltue uqarwaqw s lurfasxa wsp k byuqxw mptp ufcy qs mt kqlcn qxghzy a xwx hqvu lj bbxpukwg rxqoky lvvqiqzf cbnqlpw hszpcgo pcijlb iy x mhuqketq nqndyxhg chsb qutsy baxt tmx nqa xsfvhou vday e g fczewi bbn gxevasi rvsfove qlnvbyy wnl azwrt tviqu c',
                 'vietxl wcacbtvt rq vmjz hiyw iuxqrnr aqmxcpij rtr w mtttdj j ybwplbgh wtwuctje qqa ihcd qgdfb fvfvsqey go pgorbf zil bzhk dghym i qn naoyntv hzorwjqu xlysxg emxuvfa l rm u f vs qmsv lfylpys ttkt kjab pzfomp huqut nhfoi bxffackl vgvzgubn hdwxi i yvz iduig urc qmo xrrz nhrigfmw bk qt n yx xq ugdnzwp u']


### 1. 'Divide and Conquer' Greedy Left Alignment

In [9]:
def running_Divide_Conquer_LA_over_input_data_and_Ms(M, input_data, columns, alignedFinal):
    results_DVLA = [];
    
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            #Start processing time: 
            start_time = time.time()
            
            
            result_set = divide_conquer_alignment(input_data[i], M[z])
            
            mem_usage = memory_usage((divide_conquer_alignment, (input_data[i], M[z])))
            Mib_to_Byte = max(mem_usage)*(2**20) 
            
            #End processing time: 
            end_time = time.time() - start_time
            
            words = input_data[i].split()
            ##Get list of lines with words with one blank space separating each other
            t = printing_no_aligment(words, result_set)
            
            alignedLA = []
            
            for j in range(len(t)):
                alignedLA.append(adding_extra_blank_space_to_right(M[z], t[j]))

            total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_LA(M[z], input_data[i], alignedLA)    
            results_DVLA.append(string_result(index, len(input_data[i]) , total_cost , "Divide and Conquer GD", "LA", end_time, M[z], Mib_to_Byte));    
            index+=1; 
            alignedFinal.append(alignedLA);
            
    results_DVLA = pd.DataFrame(results_DVLA , index=None, columns=columns)
    
    return results_DVLA

In [10]:
alignedLAFinal = []
DVLA_result_1_8 = running_Divide_Conquer_LA_over_input_data_and_Ms(M, input_data_1_8, columns, alignedLAFinal)
DVLA_result_1_8.to_csv('resultsDVLA_1_8.csv', sep=',' , header=None, index=None)

### 2. 'Divide and Conquer' Greedy Right Alignment


In [11]:
def running_Divide_Conquer_RA_over_input_data_and_Ms(M, input_data, columns, alignedFinal):
    results_DVRA = [];
    
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            #Start processing time: 
            start_time = time.time()
            
            
            result_set = divide_conquer_alignment(input_data[i], M[z])
            
            mem_usage = memory_usage((divide_conquer_alignment, (input_data[i], M[z])))
            Mib_to_Byte = max(mem_usage)*(2**20) 
            
            #End processing time: 
            end_time = time.time() - start_time
            
            words = input_data[i].split()
            ##Get list of lines with words with one blank space separating each other
            t = printing_no_aligment(words, result_set)
            
            alignedRA = []
            
            for j in range(len(t)):
                alignedRA.append(adding_extra_blank_space_to_left(M[z], t[j]))

            total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_RA(M[z], input_data[i], alignedRA)    
            results_DVRA.append(string_result(index, len(input_data[i]) , total_cost , "Divide and Conquer GD", "RA", end_time, M[z], Mib_to_Byte));    
            index+=1; 
            alignedFinal.append(alignedRA);
            
    results_DVRA = pd.DataFrame(results_DVRA , index=None, columns=columns)
    
    return results_DVRA

In [12]:
alignedRAFinal = []
DVRA_result_1_8 = running_Divide_Conquer_RA_over_input_data_and_Ms(M, input_data_1_8, columns, alignedRAFinal)

### 3. 'Divide and Conquer' Greedy Center Alignment


In [13]:
def running_Divide_Conquer_CA_over_input_data_and_Ms(M, input_data, columns, alignedFinal):
    results_DVCA = [];
    
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            #Start processing time: 
            start_time = time.time()
            
            
            result_set = divide_conquer_alignment(input_data[i], M[z])
            
            mem_usage = memory_usage((divide_conquer_alignment, (input_data[i], M[z])))
            Mib_to_Byte = max(mem_usage)*(2**20) 
            
            #End processing time: 
            end_time = time.time() - start_time
            
            words = input_data[i].split()
            ##Get list of lines with words with one blank space separating each other
            t = printing_no_aligment(words, result_set)
            
            alignedCA = []
            
            for j in range(len(t)):
                alignedCA.append(adding_extra_blank_space_to_center(M[z], t[j]))

            total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_CA(M[z], input_data[i], alignedCA)   
            results_DVCA.append(string_result(index, len(input_data[i]) , total_cost , "Divide and Conquer GD", "CA", end_time, M[z], Mib_to_Byte));    
            index+=1; 
            alignedFinal.append(alignedCA);
            
    results_DVCA = pd.DataFrame(results_DVCA , index=None, columns=columns)
    
    return results_DVCA

In [14]:
alignedCAFinal = []
DVCA_result_1_8 = running_Divide_Conquer_CA_over_input_data_and_Ms(M, input_data_1_8, columns, alignedCAFinal)